In [ ]:
from typing import List
from Account import Account
from Cash import CashContext
from Clock import Clock
from Financing import Financing, FinancingContext
from Property import Property, PropertyContext
from StockInvestment import StockInvestment, StockInvestmentContext
from plotting import PlotSeries, plot

TIME_FRAME_IN_YEARS = 50

cash_context: CashContext = {
    "initial_cash": 50000,
    "monthly_salary": 2500,
    "yearly_salary_increase": 0.05,
}

stock_investment_context: StockInvestmentContext = {"expected_yield": 0.1}

financing_context: FinancingContext = {
    "interest_rate": 0.036,
    "initial_amortization_rate": 0.02,
}

# Price and rent for Eschborn, taken from the internet
property_context: PropertyContext = {
    "area": 100,
    "price_per_square_meter": 4130,
    "monthly_rent_per_square_meter": 14.49,
    "yearly_value_increase_percentage": 0.05,
    "yearly_maintenance_costs_percentage": 0.014,
    "property_purchase_tax_rate": 0.06,
    "property_purchase_notary_cost_rate": 0.015,
    "property_purchase_brokerage": 0.0357,
}

valueSeriesList: List[PlotSeries] = []
stockValueSeriesList: List[PlotSeries] = []
propValueSeriesList: List[PlotSeries] = []
financingValueSeriesList: List[PlotSeries] = []
for house_purchase_delay in range(0, 60, 10):
    legendText = (
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    valueSeries = PlotSeries(legendText)
    stockValueSeries = PlotSeries(legendText)
    propValueSeries = PlotSeries(legendText)
    financingValueSeries = PlotSeries(legendText)
    clock = Clock()
    stock_inv = StockInvestment(clock, stock_investment_context)
    prop = Property(clock, property_context)
    account = Account(clock, cash_context["initial_cash"])
    financing = None
    for i in range(TIME_FRAME_IN_YEARS * 12 + 2):
        account.deposit(
            cash_context["monthly_salary"]
            * (1 + cash_context["yearly_salary_increase"]) ** (i / 12)
        )

        stock_inv.pay_costs(account)
        prop.pay_costs(account)
        account.pay_costs(account)
        if financing is not None:
            financing.pay_costs(account)

        if i == house_purchase_delay * 12:
            required_additional_cash = (
                prop.get_total_purchase_costs() - account.balance
            )
            stock_inv.attempt_withdrawal(account, required_additional_cash)
            required_additional_cash = (
                prop.get_total_purchase_costs() - account.balance
            )
            if required_additional_cash > 0:
                financing = Financing(
                    clock, account, required_additional_cash, financing_context
                )
            prop.buy(account)

        stock_inv.deposit(account, -1)

        valueSeries.put_value((stock_inv.get_value() + prop.get_value()) / 1000000)
        stockValueSeries.put_value(stock_inv.get_value() / 1000000)
        propValueSeries.put_value(prop.get_value() / 1000000)
        financingValueSeries.put_value(
            0 if financing is None else financing.get_value()
        )
        clock.tick()
    valueSeriesList.append(valueSeries)
    stockValueSeriesList.append(stockValueSeries)
    propValueSeriesList.append(propValueSeries)
    financingValueSeriesList.append(financingValueSeries)

plot(
    {
        "title": "Total Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": valueSeriesList,
        "legend_loc": "upper left"
    }
)
plot(
    {
        "title": "Stock Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": stockValueSeriesList,
        "legend_loc": "upper left"
    }
)
plot(
    {
        "title": "Property Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": propValueSeriesList,
        "legend_loc": "upper left"
    }
)
plot(
    {
        "title": "Financing Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": financingValueSeriesList,
        "legend_loc": "lower right"
    }
)